## Things to do CV Code

1. Install OpenCV on Raaspberry pi 
2. Install X11 apps to send live feed to laptop  
    1. Use Xming X to see live feed

3. Install or create Database for tennis balls and cardboard boxes
4. Calibrate the camera to get Camera matrices
5. Create or modify existing model for Tennis ball detection
6. Get a Heatsink for Raspberry Pi 



<table>
<thead>
<tr><th>Models</th><th>Estimated FPS on Raspberry Pi 4B</th><th>Notes</th><th>Accuracy (mAP)</th></tr>
</thead>
<tbody>
<tr>
<td>Custom Algorithm (e.g., color thresholding + blob detection)</td>
<td>10 FPS</td>
<td>
- Allows the model to be tailored to the robot's conditions<br>
- More time-consuming to create model from scratch<br>
- Can be very fast but may struggle with complex scenes or varying lighting conditions
</td>
<td>Varies (typically lower than ML models)</td>
</tr>
<tr>
<td>MobileNet-SSD v2</td>
<td>1-3 FPS</td>
<td>
- Good balance of speed and accuracy<br>
- Designed for mobile and embedded devices<br>
- Widely used and well-documented
</td>
<td>22.1% mAP (COCO dataset)</td>
</tr>
<tr>
<td>YOLOv5-nano</td>
<td>0.5-2 FPS</td>
<td>
- Very small model size (~1MB)<br>
- Good for detecting small objects<br>
- May require careful optimization for Raspberry Pi
</td>
<td>25.4% mAP (COCO dataset)</td>
</tr>
<tr>
<td>EfficientDet-Lite0</td>
<td>0.5-1.5 FPS</td>
<td>
- Balanced efficiency-accuracy trade-off<br>
- Part of the EfficientDet family, known for scalability<br>
- Good documentation and TensorFlow Lite support
</td>
<td>27.5% mAP (COCO dataset)</td>
</tr>
<tr>
<td>TinyYOLOv3</td>
<td>1-3 FPS</td>
<td>
- Designed for speed on embedded devices<br>
- Smaller and faster than full YOLOv3<br>
- May miss small objects more often than larger models
</td>
<td>33.1% mAP (COCO dataset)</td>
</tr>
<tr>
<td>MobileNetV2 + SSDLite</td>
<td>1-2 FPS</td>
<td>
- Optimized version of MobileNet-SSD<br>
- Very memory-efficient<br>
- Good for resource-constrained devices
</td>
<td>22% mAP (COCO dataset)</td>
</tr>
</tbody>
</table>

predefined models may be faster if we limit what its looking for, we could use this as a safety meaasure, incase custom model doesnt work
-   I.e. Work on developing a pre developed model such as MobileNet or YOLO through TensorLite, then work on Custom Model 
- USe of Roboflow to create Datasets, or use Predefined datasets, limiting what its trained on 

### Custom Algorithm
#### Colour Based Detection 
1. Convert image to HSV colourspace
2. Define colour range of tennis balls 
3. Create binary mask using the colour range of tennis ball
4. Apply morphological operations to clean the mask 
5. Filter contours based on area and circularity

#### Edge Detection 
1. Convert images to greyscale
2. Apply Gaussian Blur to reduce noise
3. Use Canny Edge detection 
4. Apply Hough Circle transform 
5. Filter circles based on size

#### Combination of methods 
1. Use colour thereshold to create a region of interest 
2. Apply Hough Transformation only on ROI 
3. Verify using colour information 

#### Ball Tracking (Kalman Filters)
1. Initialise Kalman Filter for each ball detected 
2. Predict new positions in each frame 
3. Update predictions with new detections

##### Tips to make it faster
- Implimenting multi-threadings to parallelize detection and robot control 
-       reducing image resolution to improve processing speed
- 


### 1. Installing Dependencies

In [ ]:
# add all dependencies here for google colab and training 

# the following section of code needs to be un when operating on google colab
!pip uninstall Cython -y
!git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# copying setup files into model/research folder
%%bash 
cd models/research
protoc object_detection/protos/*.proto --python_out=.
#cp object_detection/packages/tf2/setup.py .

In [ ]:
# Modify setup.py file to install the tf-models-official repository targeted at TF v2.8.0
import re
with open('/content/models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open('/content/models/research/setup.py', 'w') as f:
    # Set fine_tune_checkpoint path
    s = re.sub('tf-models-official>=2.5.1',
               'tf-models-official==2.8.0', s)
    f.write(s)

In [ ]:
# Install the Object Detection API (NOTE: This block takes about 10 minutes to finish executing)

# Need to do a temporary fix with PyYAML because Colab isn't able to install PyYAML v5.4.1
!pip install pyyaml==5.3
!pip install /content/models/research/

# Need to downgrade to TF v2.8.0 due to Colab compatibility bug with TF v2.10 (as of 10/03/22)
!pip install tensorflow==2.8.0

# Install CUDA version 11.0 (to maintain compatibility with TF v2.8.0)
!pip install tensorflow_io==0.23.1
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget http://developer.download.nvidia.com/compute/cuda/11.0.2/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
!apt-key add /var/cuda-repo-ubuntu1804-11-0-local/7fa2af80.pub
!apt-get update && sudo apt-get install cuda-toolkit-11-0
!export LD_LIBRARY_PATH=/usr/local/cuda-11.0/lib64:$LD_LIBRARY_PATH

In [ ]:
# verifying the model by running Model Builder
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py


### 2. Dataset manipulation, getting it ready for training

df

In [ ]:
# add code for dataset manipulation 

# loading dataset


### 3. Setup Training Configuration

In [4]:
# Change the chosen_model variable to deploy different models available in the TF2 object detection zoo
chosen_model = 'ssd-mobilenet-v2-fpnlite-320'

MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    'ssd-mobilenet-v2-fpnlite-320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    },
    # The centernet model isn't working as of 9/10/22
    #'centernet-mobilenet-v2': {
    #    'model_name': 'centernet_mobilenetv2fpn_512x512_coco17_od',
    #    'base_pipeline_file': 'pipeline.config',
    #    'pretrained_checkpoint': 'centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz',
    #}
}

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [5]:
%mkdir /content/model/my_model
%cd /content/model/my_model

#downloading the pretrained model
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

[WinError 3] The system cannot find the path specified: '/content/model/my_model'
g:\Other computers\Surface\Uni Work\2024\Sem 2\ECE4191\github\ECE4191_G43_S2_2024\CV


The syntax of the command is incorrect.
'wget' is not recognized as an internal or external command,
operable program or batch file.


FileNotFoundError: [Errno 2] No such file or directory: 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

### 4. Training the data

### 5. Converting model o TensorFlow Lite ans saving it

### 6. Deployment of Model

In [ ]:
# Load the saved model and use it to detect in this bit

### 7. Quantasize the model (maybe)